In [1]:
# Import Libraries -------------------------------------------------------------------------------------------------------------------------------------------------------------
import os
import h5py
import keras
import loss
import Helper
import allMetrics
import numpy as np
import tensorflow as tf
import UNetModel_3D_simplified
import matplotlib.pyplot as plt
from keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, CSVLogger

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
ds_num = 6
train_fileName = f"/media/dro/JHSeagate/FYP/jh_fyp_work/3D_UNet/trials/numpyDatasets/numPyArrays/train_Scans/train_DS{ds_num}.hdf5"
train_maskfileName = f"/media/dro/JHSeagate/FYP/jh_fyp_work/3D_UNet/trials/numpyDatasets/numPyArrays/train_Mask_Scans/train_maskDS{ds_num}.hdf5"

train_DatasetName = f"trainScans_DataSet{ds_num}"
train_maskDatasetName = f"trainMaskScans_DataSet{ds_num}"

valid_fileName = f"/media/dro/JHSeagate/FYP/jh_fyp_work/3D_UNet/trials/numpyDatasets/numPyArrays/valid_Scans/valid_DS{ds_num}.hdf5"
valid_maskfileName = f"/media/dro/JHSeagate/FYP/jh_fyp_work/3D_UNet/trials/numpyDatasets/numPyArrays/valid_Mask_Scans/valid_maskDS{ds_num}.hdf5"

valid_DatasetName = f"validScans_DataSet{ds_num}"
valid_maskDatasetName = f"validMaskScans_DataSet{ds_num}"

test_fileName = f"/media/dro/JHSeagate/FYP/jh_fyp_work/3D_UNet/trials/numpyDatasets/numPyArrays/test_Scans/test_DS{ds_num}.hdf5"
test_maskfileName = f"/media/dro/JHSeagate/FYP/jh_fyp_work/3D_UNet/trials/numpyDatasets/numPyArrays/test_Mask_Scans/test_maskDS{ds_num}.hdf5"

test_DatasetName = f"testScans_DataSet{ds_num}"
test_maskDatasetName = f"testMaskScans_DataSet{ds_num}"

In [4]:
with h5py.File(train_fileName, 'r') as hf: # File Dir
    train_array = hf[train_DatasetName][:]
    
with h5py.File(train_maskfileName,'r') as hf:
    train_mask_array = hf[train_maskDatasetName][:]
    
with h5py.File(valid_fileName, 'r') as hf: # File Dir
    valid_array = hf[valid_DatasetName][:]
    
with h5py.File(valid_maskfileName,'r') as hf:
    valid_mask_array = hf[valid_maskDatasetName][:]
    
with h5py.File(test_fileName, 'r') as hf: # File Dir
    test_array = hf[test_DatasetName][:]
    
with h5py.File(test_maskfileName,'r') as hf:
    test_mask_array = hf[test_maskDatasetName][:]

In [7]:
print(train_array.shape)
print(train_mask_array.shape)
print('--')
print(valid_array.shape)
print(valid_mask_array.shape)
print('--')
print(test_array.shape)
print(test_mask_array.shape)

(2739, 64, 64, 96)
(2739, 64, 64, 96)
--
(1140, 64, 64, 96)
(1140, 64, 64, 96)
--
(1116, 64, 64, 96)
(1116, 64, 64, 96)


In [8]:
train_scan = np.concatenate((train_array,valid_array,test_array))
train_mask_scan = np.concatenate((train_mask_array,valid_mask_array,test_mask_array))

In [9]:
train_array = np.expand_dims(train_scan, axis=4)
train_mask_array = np.expand_dims(train_mask_scan, axis=4)

valid_array = np.expand_dims(valid_array, axis=4)
valid_mask_array = np.expand_dims(valid_mask_array, axis=4)

In [10]:
print(train_array.shape)
print(train_mask_array.shape)
print('--')
print(valid_array.shape)
print(valid_mask_array.shape)

(4995, 64, 64, 96, 1)
(4995, 64, 64, 96, 1)
--
(1140, 64, 64, 96, 1)
(1140, 64, 64, 96, 1)


In [4]:
LR = 0.001
opt = tf.keras.optimizers.Nadam(LR)

input_shape = (64,64,32,1)
num_class = 1

metrics = [allMetrics.dice_coef]



In [5]:
model = UNetModel_3D_simplified.build_unet(input_shape, n_classes = num_class)

model.compile(optimizer=opt, loss=loss.tversky_crossentropy, metrics=metrics)
print(model.summary())

sigmoid
Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 64, 64, 32,  0           []                               
                                 1)]                                                              
                                                                                                  
 conv3d_19 (Conv3D)             (None, 64, 64, 32,   896         ['input_2[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization_18 (BatchN  (None, 64, 64, 32,   128        ['conv3d_19[0][0]']              
 ormalization)                  32)                                                   

In [13]:
csv_path = '/media/dro/JHSeagate/FYP/jh_fyp_work/3D_UNet/Final/CSVLogs/3mm_5mm.csv'
model_checkpoint_path = '/media/dro/JHSeagate/FYP/jh_fyp_work/3D_UNet/Final/SavedModels/3mm_5mm.hdf5'

In [14]:
my_callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=8, mode = 'auto'),
    EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True, mode = 'auto'),
    CSVLogger(csv_path, separator=',', append=True),
    ModelCheckpoint(filepath=model_checkpoint_path,
                    monitor='val_loss',
                    mode='auto',
                    verbose=1,
                    save_best_only= True)
]
#     CustomCallBack()

In [15]:
model_name = '3mm_5mm (200 Epochs)'
# Helper.telegram_bot_sendtext(f'Model {model_name} started training')

history = model.fit(train_array,
                    train_mask_array,
                    batch_size=5,
                    epochs=100,
                    verbose=1,
                    shuffle = True,
                    validation_data=(valid_array, valid_mask_array),
                    callbacks=my_callbacks)

Epoch 1/100


2023-03-08 17:00:53.262329: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8301
2023-03-08 17:00:54.109459: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 11.0.221, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


999/999 [==============================] - 152s 143ms/step - loss: 0.8154 - dice_coef: 0.4044 - val_loss: 1.2937 - val_dice_coef: 0.0074 - lr: 0.0010
Epoch 2/100
999/999 [==============================] - 138s 138ms/step - loss: 0.6529 - dice_coef: 0.5948 - val_loss: 0.6932 - val_dice_coef: 0.4378 - lr: 0.0010
Epoch 3/100
999/999 [==============================] - 138s 138ms/step - loss: 0.6013 - dice_coef: 0.6475 - val_loss: 0.6912 - val_dice_coef: 0.5376 - lr: 0.0010
Epoch 4/100
999/999 [==============================] - 138s 138ms/step - loss: 0.5827 - dice_coef: 0.6755 - val_loss: 1.1996 - val_dice_coef: 0.0121 - lr: 0.0010
Epoch 5/100
999/999 [==============================] - 137s 137ms/step - loss: 0.5660 - dice_coef: 0.6952 - val_loss: 0.8081 - val_dice_coef: 0.4250 - lr: 0.0010
Epoch 6/100
999/999 [==============================] - 138s 138ms/step - loss: 0.5461 - dice_coef: 0.7220 - val_loss: 0.6100 - val_dice_coef: 0.6132 - lr: 0.0010
Epoch 7/100
999/999 [===================

KeyboardInterrupt: 